1) Reprojection of the layers to the corresponding CRS for each park (56 layers, 8 indices, and 7 metrics).
The indices considered are: NDVI, EVI, NDSI, NBR, LST, Albedo, NDWI, LSWI
Efas, summary of the 5th percentile year-over-year median, Percentile 95, median, sd, CV, Dmin, Dmax anuales 

- Aiguestortes 32631 
- Ordesa 32630
- Peneda 32629
- Guadarrama 32630
- Picos 32629
- SierraNieves 32630
- SierraNev 32630
- Teide 32628


In [ ]:
import os
from osgeo import gdal

#// Aiguestortes
#// Ordesa
#// Peneda
#// Guadarrama
#// Picos
#// SierraNieves
#// SierraNevada
#// Teide


park = 'Teide'

def reproyectar_raster(archivo_entrada, archivo_salida, crs):
    try:
        # Open the input raster
        raster_entrada = gdal.Open(archivo_entrada)

        # Reproject the raster
        gdal.Warp(archivo_salida, raster_entrada, dstSRS=crs)

        # Close the datasets
        raster_entrada = None

        print(f"Reprojection applied to {archivo_entrada}")
    except Exception as e:
        print(f"Error reprojecting {archivo_entrada}: {str(e)}")

# Define the input and output folder

input_folder = f'C:/Users/carlo/Desktop/EFAs/{park}'
output_folder = f'C:/Users/carlo/Desktop/EFAs/{park}/reprojected'

# Define the destination projection (EPSG 25830)
crs = 'EPSG:25830'

# Check if the output folder exists and create it if not
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Carpeta de salida creada en {output_folder}")

# Iterate over each raster file in the input folder
for archivo in os.listdir(input_folder):
    if archivo.endswith('.tif'):
        # Build the input and output file paths
        archivo_entrada = os.path.join(input_folder, archivo)
        archivo_salida = os.path.join(output_folder, archivo)
        
        # Reproject the raster file
        reproyectar_raster(archivo_entrada, archivo_salida, crs)

print("Process completed.")

Ckip and align the result layers after calculating the median and sd using R script 2.

In [ ]:
import os
import glob
import rasterio
import numpy as np
from rasterio.warp import reproject, Resampling
# from rasterio.enums import Resampling
from tqdm import tqdm
import shutil

# Function to copy the input folder structure to output
def copy_folder_structure(input_folder, output_folder):
    for dirpath, dirnames, filenames in os.walk(input_folder):
        structure = os.path.join(output_folder, os.path.relpath(dirpath, input_folder))
        os.makedirs(structure, exist_ok=True)

# Get the current directory of the script
base_path = os.getcwd()

# Define the parks
parques = ['Aiguestortes', 'Ordesa', 'Peneda', 'Guadarrama', 'Picos', 'SierraNieves', 'SierraNevada', 'Teide']

# paths for input and output folders
input_base_folder = "results"
output_base_folder = "results/Alineados"

# Base layer path
base_raster_path = "H:/.shortcut-targets-by-id/1M44eQfgiKNtbHWnOemX3695BodORG2B6/LW_SEM/WP4/EarthCul/Variables_layers/BaseLayers/{parque}/Slope.tif"
# base_raster_path = "C:/Users/carlo/Desktop/Diversity/grid/{parque}/Grid_new.tif"

for parque in parques:
    input_folder = os.path.join(base_path, input_base_folder, parque)
    output_folder = os.path.join(base_path, output_base_folder, parque)

   # Copy the input folder structure to the output
    copy_folder_structure(input_folder, output_folder)

    # Base layer route for this park
    base_raster_path_parque = base_raster_path.format(parque=parque)

    with rasterio.open(base_raster_path_parque) as base_src:
        for dirpath, dirnames, filenames in os.walk(input_folder):
            for filename in filenames:
                # Check if the file is a TIF file
                if filename.endswith('.tif'):
                    input_file = os.path.join(dirpath, filename)
                    output_file = os.path.join(output_folder, os.path.relpath(input_file, input_folder))

                    with rasterio.open(input_file) as src:
                        data = src.read(1)
                        profile = src.profile

                        # Reproject/resample the input raster to match the base raster
                        data_resampled = np.zeros_like(base_src.read(1))
                        rasterio.warp.reproject(
                            source=data,
                            destination=data_resampled,
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=base_src.transform,
                            dst_crs=base_src.crs,
                            resampling=Resampling.nearest
                        )

                        # Update the profile of the input raster to match the base raster's metadata
                        profile_resampled = src.profile.copy()
                        profile_resampled.update({
                            'transform': base_src.transform,
                            'height': data_resampled.shape[0],
                            'width': data_resampled.shape[1],
                            'crs': base_src.crs
                        })

                        # Save
                        with rasterio.open(output_file, 'w', **profile_resampled) as dst:
                            dst.write(data_resampled, 1)

In [ ]:
import os
from osgeo import gdal

# Define the list of parks
parques = ['Aiguestortes', 'Ordesa', 'Peneda', 'Guadarrama', 'Picos', 'SierraNieves', 'SierraNevada', 'Teide']

# Path to the mask shapefile
base_mask_folder = 'H:/.shortcut-targets-by-id/1M44eQfgiKNtbHWnOemX3695BodORG2B6/LW_SEM/WP4/EarthCul/Variables_layers/BaseLayers'

# Base folder for input and output rasters
base_input_folder = "C:/Users/carlo/Desktop/EFAs/results/Alineados"
base_output_folder = "C:/Users/carlo/Desktop/EFAs/results/Clip"

# Function to copy the input folder structure to output
def copy_folder_structure(input_folder, output_folder):
    for dirpath, dirnames, filenames in os.walk(input_folder):
        structure = os.path.join(output_folder, os.path.relpath(dirpath, input_folder))
        os.makedirs(structure, exist_ok=True)

# Iterate over the parks
for parque in parques:
    input_parque_folder = os.path.join(base_input_folder, parque)
    output_parque_folder = os.path.join(base_output_folder, parque)
    mask_shapefile = os.path.join(base_mask_folder, parque, 'AOI_3035.shp')

    # Create the park output folder if it does not exist
    os.makedirs(output_parque_folder, exist_ok=True)

    # Copy the folder structure from input to output park
    copy_folder_structure(input_parque_folder, output_parque_folder)

    # Function to clip rasters
    def recortar_raster(input_raster, output_raster):
        gdal.Warp(output_raster, input_raster, cutlineDSName=mask_shapefile, cropToCutline=True, dstNodata=-9999)

    # Iterate over each raster file in the input park folder
    for dirpath, dirnames, filenames in os.walk(input_parque_folder):
        for filename in filenames:
            if filename.endswith('.tif'):
                input_raster = os.path.join(dirpath, filename)
                output_raster = os.path.join(output_parque_folder, os.path.relpath(input_raster, input_parque_folder))
                
                # clipthe raster using gdal.Warp
                recortar_raster(input_raster, output_raster)


gdal.Warp(output_raster, input_raster, cutlineDSName=mask_shapefile, cropToCutline=True, dstNodata=-9999)


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000002B596988690> >